# 📚 Agricultural PDF to Text Extraction with Chandra OCR

This notebook extracts text from agricultural textbook PDFs using **Chandra**, a state-of-the-art OCR model.

## Features:
- Complex tables and forms
- Handwriting recognition
- Mathematical formulas (LaTeX)
- Multi-column layouts
- 40+ languages

## Output Formats:
| Format | Best For | Description |
|--------|----------|-------------|
| **JSONL** | LLM Fine-tuning ✅ | One JSON object per line, easy to process |
| Markdown | Human review | Readable format with formatting |
| JSON | API/structured data | Single file, harder to stream |
| CSV | Tabular analysis | Limited for long text |
| TXT | Simple pretraining | No structure/metadata |

## 🔧 Step 1: Install Dependencies

In [ ]:
# Install Chandra OCR and dependencies
!pip install chandra-ocr pymupdf pillow tqdm -q

## 📁 Step 2: Configure Paths

In [ ]:
from pathlib import Path
import os

# ======= CONFIGURE THESE PATHS =======
PDF_DIR = Path("../data/book")  # Directory containing your PDFs
OUTPUT_DIR = Path("../data/extracted")  # Output directory
# =====================================

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# List all PDFs
pdf_files = list(PDF_DIR.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files:")
for pdf in pdf_files:
    size_mb = pdf.stat().st_size / (1024 * 1024)
    print(f"  - {pdf.name} ({size_mb:.1f} MB)")

## 🔍 Step 3: Analyze PDF Types

In [ ]:
import fitz  # PyMuPDF

def analyze_pdf(pdf_path, sample_pages=5):
    """Check if PDF is digital (has text) or scanned (needs OCR)."""
    doc = fitz.open(pdf_path)
    total_chars = 0
    pages_to_check = min(sample_pages, len(doc))
    
    for i in range(pages_to_check):
        text = doc[i].get_text().strip()
        total_chars += len(text)
    
    avg_chars = total_chars / pages_to_check if pages_to_check > 0 else 0
    is_scanned = avg_chars < 100
    
    result = {
        "path": pdf_path,
        "total_pages": len(doc),
        "avg_chars_per_page": avg_chars,
        "needs_ocr": is_scanned,
    }
    doc.close()
    return result

# Analyze all PDFs
print("PDF Analysis:")
print("-" * 60)
pdf_analysis = []
for pdf in pdf_files:
    info = analyze_pdf(pdf)
    pdf_analysis.append(info)
    status = "🔴 SCANNED (needs OCR)" if info["needs_ocr"] else "🟢 DIGITAL (fast)"
    print(f"{pdf.name}: {info['total_pages']} pages | {status}")

## ⚡ Step 4A: Fast Extraction (Digital PDFs)

In [ ]:
from tqdm.notebook import tqdm
import json

def extract_digital_pdf(pdf_path, output_dir):
    """Extract text from digital PDFs without OCR."""
    doc = fitz.open(pdf_path)
    pdf_name = Path(pdf_path).stem
    records = []
    
    for i, page in enumerate(tqdm(doc, desc=f"Extracting {pdf_name}")):
        text = page.get_text("text").strip()
        if text:
            records.append({
                "text": text,
                "source": pdf_name,
                "page": i + 1,
                "total_pages": len(doc)
            })
    
    doc.close()
    
    # Save as JSONL (best for fine-tuning)
    jsonl_path = output_dir / f"{pdf_name}.jsonl"
    with open(jsonl_path, 'w', encoding='utf-8') as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    # Save as Markdown (for preview)
    md_path = output_dir / f"{pdf_name}.md"
    with open(md_path, 'w', encoding='utf-8') as f:
        for record in records:
            f.write(f"<!-- Page {record['page']} -->\n\n{record['text']}\n\n---\n\n")
    
    print(f"Saved {len(records)} pages: {jsonl_path}")
    return records

# Extract digital PDFs
digital_pdfs = [info for info in pdf_analysis if not info["needs_ocr"]]
for info in digital_pdfs:
    extract_digital_pdf(info['path'], OUTPUT_DIR)

## 🤖 Step 4B: Chandra OCR (Scanned PDFs)

Use Chandra for high-quality OCR on scanned documents.

In [ ]:
# Using Chandra CLI (recommended for batch processing)
scanned_pdfs = [info for info in pdf_analysis if info["needs_ocr"]]

if scanned_pdfs:
    print("Processing scanned PDFs with Chandra OCR...")
    print("This may take several minutes per document.\n")
    
    for info in scanned_pdfs:
        pdf_path = str(info['path'])
        output_path = str(OUTPUT_DIR)
        
        print(f"Processing: {info['path'].name}")
        # Use HuggingFace method (local, no server needed)
        !chandra "{pdf_path}" "{output_path}" --method hf --no-images
else:
    print("No scanned PDFs found. All PDFs extracted in Step 4A.")

## 📊 Step 5: Convert Markdown to JSONL

In [ ]:
import re

def markdown_to_jsonl(md_path):
    """Convert markdown output to JSONL format."""
    md_path = Path(md_path)
    
    with open(md_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Split by page markers
    if '<!-- Page' in content:
        pages = re.split(r'<!-- Page \d+ -->', content)
        pages = [p.strip().rstrip('---').strip() for p in pages if p.strip()]
    else:
        pages = [content.strip()]
    
    source = md_path.stem
    records = []
    
    for i, text in enumerate(pages):
        if text:
            records.append({
                "text": text,
                "source": source,
                "page": i + 1,
                "total_pages": len(pages)
            })
    
    jsonl_path = md_path.with_suffix('.jsonl')
    with open(jsonl_path, 'w', encoding='utf-8') as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    print(f"Converted {len(records)} pages: {jsonl_path}")
    return records

# Convert all markdown files
md_files = list(OUTPUT_DIR.glob("**/*.md"))
for md_file in md_files:
    markdown_to_jsonl(md_file)

## 🔗 Step 6: Merge All Files into Single Dataset

In [ ]:
def merge_jsonl_files(input_dir, output_file):
    """Merge all JSONL files into a single dataset."""
    input_dir = Path(input_dir)
    jsonl_files = list(input_dir.glob("**/*.jsonl"))
    
    all_records = []
    for jsonl_path in jsonl_files:
        if jsonl_path.name == Path(output_file).name:
            continue  # Skip output file if it exists
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    all_records.append(json.loads(line))
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for record in all_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    print(f"\nMerged Dataset:")
    print(f"  Total records: {len(all_records)}")
    print(f"  Sources: {len(set(r['source'] for r in all_records))}")
    print(f"  Output: {output_file}")
    
    return all_records

DATASET_FILE = OUTPUT_DIR / "agricultural_textbooks.jsonl"
all_data = merge_jsonl_files(OUTPUT_DIR, DATASET_FILE)

## 📋 Step 7: Preview Data

In [ ]:
# Preview first 3 records
with open(DATASET_FILE, 'r', encoding='utf-8') as f:
    samples = [json.loads(line) for i, line in enumerate(f) if i < 3]

for i, sample in enumerate(samples):
    print(f"{'='*60}")
    print(f"Record {i+1}: {sample['source']} - Page {sample['page']}")
    print(f"{'='*60}")
    preview = sample['text'][:800]
    print(preview + ("..." if len(sample['text']) > 800 else ""))
    print()

---

# 📖 Dataset Format Recommendations for LLM Fine-Tuning

## Why JSONL is Best:

1. **Streaming**: Process line-by-line without loading entire file
2. **Metadata**: Store source, page numbers, etc. with each record
3. **Flexibility**: Easy to filter, sample, or split
4. **Standard**: Used by OpenAI, HuggingFace, and most fine-tuning frameworks

## Next Steps for Fine-Tuning:

1. **Continued Pre-training**: Use raw JSONL as-is
2. **Instruction Tuning**: Convert to Q&A pairs (see Step 8 below)
3. **RAG**: Create embeddings and store in vector database

## 💡 Step 8: Create Q&A Training Format (Optional)

In [ ]:
def create_qa_format(jsonl_path, output_path, format_type="chatml"):
    """Convert raw text to Q&A training format.
    
    format_type:
        - 'chatml': ChatML format for OpenAI/Llama fine-tuning
        - 'alpaca': Alpaca instruction format
    """
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        records = [json.loads(line) for line in f if line.strip()]
    
    output_records = []
    
    for r in records:
        if format_type == "chatml":
            # ChatML format (OpenAI, Llama-3, etc.)
            output_records.append({
                "messages": [
                    {"role": "system", "content": "You are an agricultural expert. Answer questions based on the provided context."},
                    {"role": "user", "content": f"Context from {r['source']} (page {r['page']}):\n\n{r['text'][:3000]}\n\n[Add your question here]"},
                    {"role": "assistant", "content": "[Add your answer here - requires annotation]"}
                ]
            })
        elif format_type == "alpaca":
            # Alpaca instruction format
            output_records.append({
                "instruction": "Answer the following agricultural question based on the context.",
                "input": f"Context: {r['text'][:3000]}\n\nQuestion: [Add question]",
                "output": "[Add answer - requires annotation]"
            })
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for record in output_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    print(f"Created {len(output_records)} training examples: {output_path}")
    print("\nNOTE: You need to add actual Q&A pairs. Consider:")
    print("  1. Manual annotation")
    print("  2. Use GPT-4 to generate Q&A pairs")
    print("  3. Extract questions from textbook exercises")

# Optional: Create training format templates
# create_qa_format(DATASET_FILE, OUTPUT_DIR / "training_chatml.jsonl", "chatml")
# create_qa_format(DATASET_FILE, OUTPUT_DIR / "training_alpaca.jsonl", "alpaca")

## ✅ Summary

Your extracted dataset is saved at: `data/extracted/agricultural_textbooks.jsonl`

### JSONL Record Format:
```json
{"text": "...", "source": "weed-control", "page": 1, "total_pages": 100}
```

### Usage with Popular Frameworks:
```python
# HuggingFace datasets
from datasets import load_dataset
dataset = load_dataset('json', data_files='agricultural_textbooks.jsonl')

# Direct loading
import json
with open('agricultural_textbooks.jsonl') as f:
    data = [json.loads(line) for line in f]
```